In [7]:
import sys, os
from pathlib import Path
import yaml
import json

PROJECT_ROOT = Path(r"C:\Users\quantbase\Desktop\sydata")
SRC = PROJECT_ROOT / "src"
DATA_ROOT = Path(r"C:\Users\quantbase\Desktop\marketdata")
MANIFEST = DATA_ROOT / "meta" / "symbols.yml"

# Make `from sydata...` importable
sys.path.insert(0, str(SRC))

# Make relative paths (scripts/, etc.) resolve predictably
os.chdir(str(PROJECT_ROOT))

print("python:", sys.executable)
print("cwd:", Path.cwd())
print("sys.path[0]:", sys.path[0])
print("SRC exists:", SRC.exists())


python: c:\Users\quantbase\.conda\envs\sydata-311\python.exe
cwd: C:\Users\quantbase\Desktop\sydata
sys.path[0]: C:\Users\quantbase\Desktop\sydata\src
SRC exists: True


In [12]:
import os, sys, subprocess
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\quantbase\Desktop\sydata")
SRC = PROJECT_ROOT / "src"
DATA_ROOT = Path(r"C:\Users\quantbase\Desktop\marketdata")
MANIFEST  = DATA_ROOT / "meta" / "symbols.yml"

env = os.environ.copy()
env["PYTHONPATH"] = str(SRC)

cmd = [
    sys.executable, "scripts/ingest_um_premium_index_klines.py",
    "--data-root", str(DATA_ROOT),
    "--manifest", str(MANIFEST),
    "--basket", "core_major",
    "--intervals", "1h",
    "--start", "2025-01-01",
    "--end", "2025-12-31",
]

cp = subprocess.run(cmd, cwd=str(PROJECT_ROOT), env=env, capture_output=True, text=True)
cp.returncode, cp.stdout[-2000:], cp.stderr[-4000:]


(0, '{\n  "ok": 84,\n  "missing_archive_file": 0,\n  "total": 84\n}\n', '')

In [15]:
import pandas as pd
from pathlib import Path

base = DATA_ROOT / "raw" / "binance" / "um_premium_index_klines"
parts = sorted(base.rglob("part-2025-01.parquet"))
len(parts), [str(p) for p in parts[:10]]


(7,
 ['C:\\Users\\quantbase\\Desktop\\marketdata\\raw\\binance\\um_premium_index_klines\\symbol=ADA-USDT\\interval=1h\\part-2025-01.parquet',
  'C:\\Users\\quantbase\\Desktop\\marketdata\\raw\\binance\\um_premium_index_klines\\symbol=BNB-USDT\\interval=1h\\part-2025-01.parquet',
  'C:\\Users\\quantbase\\Desktop\\marketdata\\raw\\binance\\um_premium_index_klines\\symbol=BTC-USDT\\interval=1h\\part-2025-01.parquet',
  'C:\\Users\\quantbase\\Desktop\\marketdata\\raw\\binance\\um_premium_index_klines\\symbol=ETH-USDT\\interval=1h\\part-2025-01.parquet',
  'C:\\Users\\quantbase\\Desktop\\marketdata\\raw\\binance\\um_premium_index_klines\\symbol=LINK-USDT\\interval=1h\\part-2025-01.parquet',
  'C:\\Users\\quantbase\\Desktop\\marketdata\\raw\\binance\\um_premium_index_klines\\symbol=SOL-USDT\\interval=1h\\part-2025-01.parquet',
  'C:\\Users\\quantbase\\Desktop\\marketdata\\raw\\binance\\um_premium_index_klines\\symbol=XRP-USDT\\interval=1h\\part-2025-01.parquet'])

In [16]:
p = parts[0]
df = pd.read_parquet(p)
df.shape, df.columns.tolist(), df[["ts","open","close"]].head(5), df["ts"].min(), df["ts"].max()


((744, 12),
 ['ts',
  'open_time',
  'close_time',
  'open',
  'high',
  'low',
  'close',
  'symbol',
  'venue_symbol',
  'interval',
  'venue',
  'dataset'],
                          ts      open     close
 0 2025-01-01 00:00:00+00:00 -0.000243  0.000000
 1 2025-01-01 01:00:00+00:00 -0.000142  0.000008
 2 2025-01-01 02:00:00+00:00  0.000000  0.000000
 3 2025-01-01 03:00:00+00:00 -0.000019 -0.000236
 4 2025-01-01 04:00:00+00:00 -0.000186 -0.000033,
 Timestamp('2025-01-01 00:00:00+0000', tz='UTC'),
 Timestamp('2025-01-31 23:00:00+0000', tz='UTC'))

In [17]:
expected = 31 * 24
df.shape[0], expected


(744, 744)